In [1]:
import os
import os.path
import glob
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold                                                                                                                       
from sklearn.metrics import confusion_matrix,accuracy_score
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import sklearn_evaluation.plot as skplot
from sklearn.model_selection import learning_curve

import matplotlib.pyplot as plt

import numpy as np
np.random.seed(1)

from keras.preprocessing.image import img_to_array
from keras.utils import np_utils
from keras.preprocessing import image
from keras.applications.densenet import DenseNet169,preprocess_input

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
imagedir = "Datasets/ByFamilyPortexEntropy"

In [3]:
cur_dir = os.getcwd()
os.chdir(imagedir)  # the parent folder with sub-folders

# Get number of samples per family
list_fams = sorted(os.listdir(os.getcwd()), key=str.lower)  # vector of strings with family names
no_imgs = []  # No. of samples per family
for i in range(len(list_fams)):
    os.chdir(list_fams[i])
    len1 = len(glob.glob('*.png'))  # assuming the images are stored as 'png'
    no_imgs.append(len1)
    os.chdir('..')
num_samples = np.sum(no_imgs)  # total number of all samples

# Compute the labels
y = np.zeros(num_samples)
pos = 0
label = 0
for i in no_imgs:
    print ("Label:%2d\tFamily: %15s\tNumber of images: %d" % (label, list_fams[label], i))
    for j in range(i):
        y[pos] = label
        pos += 1
    label += 1
num_classes = label

# Compute the features
width, height,channels = (224,224,3)
X = np.zeros((num_samples, width, height, channels))
cnt = 0
list_paths = [] # List of image paths
print("Processing images ...")
for i in range(len(list_fams)):
    for img_file in glob.glob(list_fams[i]+'/*.png'):
        #print("[%d] Processing image: %s" % (cnt, img_file))
        list_paths.append(os.path.join(os.getcwd(),img_file))
        img = image.load_img(img_file, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        x = preprocess_input(x)
        X[cnt] = x
        cnt += 1
print("Images processed: %d" %(cnt))

os.chdir(cur_dir)

Label: 0	Family:         allaple	Number of images: 362
Label: 1	Family:           alman	Number of images: 325
Label: 2	Family:          autoit	Number of images: 261
Label: 3	Family:            daws	Number of images: 466
Label: 4	Family:            delf	Number of images: 359
Label: 5	Family:         gamarue	Number of images: 259
Label: 6	Family:          ibryte	Number of images: 347
Label: 7	Family:          loring	Number of images: 285
Label: 8	Family:          mydoom	Number of images: 578
Label: 9	Family:          qukart	Number of images: 253
Label:10	Family:          ramnit	Number of images: 506
Label:11	Family:          sality	Number of images: 1401
Label:12	Family:          simbot	Number of images: 1148
Label:13	Family:       softpulse	Number of images: 912
Label:14	Family:          viking	Number of images: 183
Label:15	Family:         virlock	Number of images: 373
Label:16	Family:          vobfus	Number of images: 405
Label:17	Family:          wapomi	Number of images: 345
Label:18

In [4]:
X.shape

(10136, 224, 224, 3)

In [5]:
# Creating base_model
image_shape = (224, 224, 3)                                                                                                                                                                                                                                                                                            
base_model = DenseNet169(weights='imagenet', input_shape=image_shape, include_top=False, pooling='avg')

In [6]:
print("Extracting bottleneck features from convolutional layers ...")
bnfeatures = base_model.predict(X)

Extracting bottleneck features from convolutional layers ...


In [7]:
bnfeatures.shape

(10136, 1, 1, 1664)

In [8]:
bnfeatures = np.reshape(bnfeatures,(bnfeatures.shape[0],-1))

In [9]:
bnfeatures.shape

(10136, 1664)

In [10]:
# Create stratified k-fold subsets                                                                                                                                        
kfold = 10  # no. of folds                                                                 
skf = StratifiedKFold(kfold, shuffle=True,random_state=1)

In [11]:
# Training top_model
C_range = np.logspace(-2, 10, 13)
param_grid = dict(C=C_range)

top_model = GridSearchCV(svm.LinearSVC(), param_grid=param_grid, cv=skf, verbose=10, n_jobs=-1)
top_model.fit(bnfeatures,y)  # Training 

Fitting 10 folds for each of 13 candidates, totalling 130 fits
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.01 ..........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...........................................................
[CV] C=0.1 ...

[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:  1.7min


[CV] ........................... C=0.01, score=0.934589, total= 1.7min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.927794, total= 1.8min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.936024, total= 1.8min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.914706, total= 2.1min
[CV] C=1.0 ...........................................................
[CV] ........................... C=0.01, score=0.929550, total= 2.2min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.915039, total= 2.3min
[CV] C=10.0 ..........................................................
[CV] ........................... C=0.01, score=0.910433, total= 2.5min
[CV] C=10.0 ..........................................................
[CV] .

[Parallel(n_jobs=-1)]: Done  13 tasks      | elapsed:  4.8min


[CV] ............................ C=0.1, score=0.928150, total= 4.8min
[CV] C=10.0 ..........................................................
[CV] ............................ C=0.1, score=0.913690, total= 4.9min
[CV] ............................ C=0.1, score=0.927794, total= 4.9min
[CV] C=10.0 ..........................................................
[CV] C=10.0 ..........................................................
[CV] ............................ C=1.0, score=0.903320, total= 5.6min
[CV] C=100.0 .........................................................
[CV] ............................ C=1.0, score=0.899606, total= 5.8min
[CV] C=100.0 .........................................................
[CV] ............................ C=1.0, score=0.913894, total= 5.9min
[CV] C=100.0 .........................................................
[CV] ............................ C=0.1, score=0.913347, total= 6.2min
[CV] C=100.0 .........................................................
[CV] .

...

[CV] .................... C=100000000.0, score=0.911531, total= 6.2min
[CV] ................... C=1000000000.0, score=0.911417, total= 4.9min
[CV] ................... C=1000000000.0, score=0.893849, total= 4.7min
[CV] ................... C=1000000000.0, score=0.909990, total= 4.9min
[CV] ................... C=1000000000.0, score=0.907830, total= 4.9min
[CV] ................... C=1000000000.0, score=0.900391, total= 5.9min
[CV] ................... C=1000000000.0, score=0.911531, total= 4.7min
[CV] ................... C=1000000000.0, score=0.897260, total= 6.0min
[CV] .................. C=10000000000.0, score=0.885827, total= 4.0min
[CV] .................. C=10000000000.0, score=0.906863, total= 4.1min
[CV] .................. C=10000000000.0, score=0.897410, total= 3.8min
[CV] .................. C=10000000000.0, score=0.903865, total= 4.0min
[CV] ................... C=1000000000.0, score=0.901394, total= 5.1min
[CV] .................. C=10000000000.0, score=0.900391, total= 5.1min


[Parallel(n_jobs=-1)]: Done 125 out of 130 | elapsed: 29.7min remaining:  1.2min


[CV] .................. C=10000000000.0, score=0.897260, total= 5.1min
[CV] .................. C=10000000000.0, score=0.911531, total= 4.6min
[CV] .................. C=10000000000.0, score=0.907023, total= 4.9min
[CV] .................. C=10000000000.0, score=0.907480, total= 5.0min
[CV] .................. C=10000000000.0, score=0.892857, total= 4.8min


[Parallel(n_jobs=-1)]: Done 130 out of 130 | elapsed: 29.9min finished


GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=1, shuffle=True),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=-1,
       param_grid={'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03, 1.e+04, 1.e+05,
       1.e+06, 1.e+07, 1.e+08, 1.e+09, 1.e+10])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=10)

In [12]:
top_model.cv_results_

{'mean_fit_time': array([121.86959558, 333.49289069, 317.6054611 , 322.97169292,
        361.32669325, 327.00107284, 302.29063802, 356.16849017,
        315.09835498, 332.98707364, 328.84654443, 304.72064414,
        272.18727257]),
 'mean_score_time': array([0.13406684, 0.12221558, 0.09696369, 0.05805962, 0.07563586,
        0.06376305, 0.04619005, 0.06499097, 0.05610654, 0.0707824 ,
        0.07432449, 0.03146534, 0.03556001]),
 'mean_test_score': array([0.9234412 , 0.9198895 , 0.90913575, 0.90459747, 0.90321626,
        0.9043015 , 0.90390687, 0.90400552, 0.90301894, 0.90459747,
        0.9043015 , 0.90262431, 0.90104578]),
 'mean_train_score': array([0.97365809, 0.99361994, 0.99773093, 0.99838849, 0.99688722,
        0.99836653, 0.99834488, 0.99755561, 0.99763292, 0.99814723,
        0.99834464, 0.9977741 , 0.99752207]),
 'param_C': masked_array(data=[0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0,
                    1000000.0, 10000000.0, 100000000.0, 1000000000.0,
      

In [13]:
top_model.grid_scores_

/usr/local/lib/python3.5/dist-packages/sklearn/model_selection/_search.py:667: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


[mean: 0.92344, std: 0.00816, params: {'C': 0.01},
 mean: 0.91989, std: 0.00662, params: {'C': 0.1},
 mean: 0.90914, std: 0.00691, params: {'C': 1.0},
 mean: 0.90460, std: 0.00678, params: {'C': 10.0},
 mean: 0.90322, std: 0.00786, params: {'C': 100.0},
 mean: 0.90430, std: 0.00788, params: {'C': 1000.0},
 mean: 0.90391, std: 0.00987, params: {'C': 10000.0},
 mean: 0.90401, std: 0.00760, params: {'C': 100000.0},
 mean: 0.90302, std: 0.00982, params: {'C': 1000000.0},
 mean: 0.90460, std: 0.00766, params: {'C': 10000000.0},
 mean: 0.90430, std: 0.00679, params: {'C': 100000000.0},
 mean: 0.90262, std: 0.00806, params: {'C': 1000000000.0},
 mean: 0.90105, std: 0.00745, params: {'C': 10000000000.0}]

In [14]:
print("Best parameters: %s - score: %0.4f" % (top_model.best_params_, top_model.best_score_))

Best parameters: {'C': 0.01} - score: 0.9234
